# Fit RC params of retina - Preprocessing

Preprocesse the experimentally recorded currents with and without retina.

Fit sinusoids to the data to make the further processing simpler.

Estimate the voltage drop over the electrical double layer EDL, and prepare the parameter optimization of the eletrical properties of the retina.

# Imports

In [ ]:
import pandas as pd
import itertools
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import minimize

In [ ]:
from scipy.io import loadmat

In [ ]:
import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', 'pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import data_utils

# Helper functions

In [ ]:
def sin_A_phi_time_f(A, phi, time, f):
    return A*np.sin(2*np.pi*time*f+phi/180*np.pi)

# Preprocess data

In [ ]:
data_folder = os.path.join('..', 'experimental_data', 'data_sinusoidal_currents')

In [ ]:
ws = ['w', 'wo']
fs = [25, 40]

In [ ]:
os.listdir(data_folder)

In [ ]:
data_utils.make_dir('data_preprocessed')
os.listdir('data_preprocessed')

## Load recorded currents

In [ ]:
raw_cur = {'w': {}, 'wo': {}}

raw_cur['wo'][25] = loadmat(f'{data_folder}/2019-03-21p1_current_25Hz_ohne retina.mat')['data'][0][0][1]
raw_cur['wo'][40] = loadmat(f'{data_folder}/2019-03-21p1_current_40Hz_ohne retina.mat')['data'][0][0][1]

raw_cur['w'][25] = loadmat(f'{data_folder}/2019-03-21p1_current_25Hz_mit retina.mat')['data'][0][0][1]
raw_cur['w'][40] = loadmat(f'{data_folder}/2019-03-21p1_current_40Hz_mit retina.mat')['data'][0][0][1]

In [ ]:
cur_time = {}
cur_time[25] = np.linspace(0,1/25,raw_cur['wo'][25].shape[0])
cur_time[40] = np.linspace(0,1/40,raw_cur['wo'][40].shape[0])

## Set and save used voltage amplitudes

In [ ]:
V_amps = {}
V_amps[25] = [30, 100, 200, 300, 400, 500, 600]
V_amps[40] = [15, 50, 100, 150, 200, 250, 300]

data_utils.save_var(V_amps, 'data_preprocessed/V_amps.pkl')

## Table of raw currents and voltages

In [ ]:
cur = {'w': {}, 'wo': {}}

for w, f in itertools.product(ws, fs):
    sort_idx = np.argsort(np.max(raw_cur[w][f], axis=0))

    cur[w][f] = pd.DataFrame(np.concatenate([np.atleast_2d(cur_time[f]).T, raw_cur[w][f][:,sort_idx]], axis=1))
    cur[w][f].columns = ['Time'] + [str(i) + ' mV' for i in V_amps[f]]

In [ ]:
cur[w][40].head()

In [ ]:
data_utils.save_var(cur, 'data_preprocessed/raw_currents.pkl')
data_utils.save_var(cur_time, 'data_preprocessed/cur_time.pkl')

## Plot raw currents

In [ ]:
for w, f  in itertools.product(ws, fs):
    plt.figure(figsize=(12,1.5))
    plt.title(str(w) + ' ' + str(f))
    for i_V, V in enumerate(V_amps[f]):
        plt.plot(cur_time[f], cur[w][f][str(V) + " mV"]*1e6, 'r', label='current' if V == 15 or V == 30 else '')
        plt.legend()

## Fit sinusoids to measured currents

In [ ]:
def fit_sin2raw_current(x, f, target_current):
    A   = x[0]
    phi = x[1] 
    return np.sum((sin_A_phi_time_f(A, phi, cur_time[f], f) - 1e6*target_current)**2)

np.random.seed(13454642)

# Fit.
fit_sin_params = {w: {f: {V: None for V in V_amps[f]} for f in fs} for w in ws}
for w, f  in itertools.product(ws, fs):
    for V in V_amps[f]:
        A0 = np.max(np.abs((cur[w][f][str(V) + " mV"])))*1e6
        phi0 = (cur_time[f][np.where(cur[w][f][str(V) + " mV"] < 0)[0][0]] * f) * 180

        best_fit_loss = np.inf
        best_fit = None
        for _ in range(10):
            fit = minimize(
                fit_sin2raw_current, args=(f, cur[w][f][str(V) + " mV"]),
                x0=[A0*np.random.uniform(0.5, 2), phi0*np.random.uniform(0.5, 2)],
                bounds=[(A0*0.5, A0*2), (-90, 90)]
            )

            if fit.fun < best_fit_loss:
                best_fit_loss = fit.fun
                best_fit = fit.x
                
        fit_sin_params[w][f][V] = best_fit
        fit_sin_params[w][f][V][0] /= 1e6

### Plot sinus fits and data

In [ ]:
for w, f in itertools.product(ws, fs):
    plt.figure(figsize=(12,1.5))
    plt.title(str(w) + ' ' + str(f))
    for i_V, V in enumerate(V_amps[f]):
        plt.plot(cur_time[f], cur[w][f][str(V) + " mV"]*1e6,\
                 'r', label='current' if V == 15 or V == 30 else '')
        plt.plot(
            cur_time[f],
            sin_A_phi_time_f(fit_sin_params[w][f][V][0], fit_sin_params[w][f][V][1], cur_time[f], f)*1e6,
            'k', label='fit' if V == 15  or V == 30 else ''
        )
        plt.legend()

In [ ]:
plt.figure(figsize=(12,3))
plt.title('Phis from sinus fits')
for w, f in itertools.product(ws, fs):
    phis = np.array(list(fit_sin_params[w][f].values()))[:,1]
    Vs = np.array(V_amps[f])/1000

    plt.plot(Vs, phis, '-*', label=str(w) + ' ' + str(f))
plt.legend()
plt.xlabel('V')
plt.ylabel('phase')
plt.show()

In [ ]:
plt.figure(figsize=(12,3))
plt.title('Amplitudes from sinus fits')
for w, f in itertools.product(ws, fs):
    As = np.array(list(fit_sin_params[w][f].values()))[:,0] * 1e6
    Vs = np.array(V_amps[f])/1000

    plt.plot(Vs, As, '-*', label=str(w) + ' ' + str(f))
plt.legend()
plt.xlabel('V')
plt.ylabel('I [uA]')
plt.show()

### Save to file

In [ ]:
data_utils.save_var(fit_sin_params, 'data_preprocessed/raw_currents_sinus_fits_params.pkl')

# Create COMSOL stimulation currents.
Use fitted sinusoids instead of the measured currents.

## Create stimulus currents

In [ ]:
data_utils.make_dir('data_preprocessed/COMSOL')

In [ ]:
# Write currents to file.
N = 10 # Number of repetitions.
dt = 2e-5 # Time step.

for w, f in itertools.product(ws, fs):
    for i_V, V in enumerate(V_amps[f]):
        if w == 'w':
            df_name = 'ret' + str(f) + 'Hz'
        else:
            df_name = 'ames' + str(f) + 'Hz'
        # Outputfile.
        file = 'data_preprocessed/COMSOL/' + df_name + '_' + str(V) + '.txt'
        print("Save to \"" + file + "\"")

        # Create currents.
        time = np.arange(0,int(N/f/dt))*dt
        current = sin_A_phi_time_f(fit_sin_params[w][f][V][0], fit_sin_params[w][f][V][1], time, f) 

        pd.DataFrame({'Time': time*1e3, 'Current': current*1e6}).to_csv(file, index=False, header=None)

In [ ]:
# This string can be used in COMSOL to switch between different currents.
current_str = ""
for f in fs:
    for I, V_amp in enumerate(V_amps[f]):
        current_str += "(I==" + str(V_amp) +  ")*(f==" + str(f) +  ")*I_" + str(f) + "_" + str(V_amp) + "(t) + "
current_str = current_str[:-3]
print(current_str)

## ! Manually run COMSOL

In [ ]:
data_utils.make_dir('V_ames')

- Open **step1_flat_wo_exp_currents.mph** in COMSOL. It is available in the folder **COMSOL2python_COMSOL**.
- In "Global Defintions" and ensure that the paths to the currents are correct.
- In "Results/Export/V ele plot", ensure that the output path is correct.

Run Experiment without Retina and electrical circuit to voltage drop over ames.

In [ ]:
input('Confirm that COMSOL output is ready!')

In [ ]:
V_ames_wo_file = 'V_ames/V_ames_wo.txt'
assert os.path.isfile(V_ames_wo_file), 'The COMSOL export failed. Make sure the paths are correct.'

## Load COMSOL data (voltage drop over ames).

In [ ]:
comsol_dt = 1e-5

# Load comsol simulation data.
V_ames_raw = pd.read_csv(V_ames_wo_file, comment='%', header=None, delim_whitespace=True)

# Split.
start_idxs = np.append(np.where(V_ames_raw.iloc[:,0] == 0)[0], -1)

# Rearange raw data.
V_ames = {25: {}, 40: {}}
i = 0
for f in fs:
    for V in V_amps[f]:
        V_ames[f][V] = V_ames_raw.iloc[start_idxs[i]:start_idxs[i+1],:].values
        
        # Remove first period.
        V_ames[f][V] = V_ames[f][V][int(1/f/comsol_dt):,:]
        V_ames[f][V][:,0] -= V_ames[f][V][0,0]
        
        i += 1

### Plot.

In [ ]:
plt.figure(figsize=(12,6))
for idx_f, f in enumerate(fs):
    ax = plt.subplot(2,2,idx_f+1)
    plt.title(str(f) + ' Hz')
    ax2 = ax.twinx()
    for i_V, V in enumerate(V_amps[f]):
        time = V_ames[f][V][:,0]
        ax.plot(time, V_ames[f][V][:,1], c='C'+str(i_V), alpha=0.5, label='V' + str(V))
        current = sin_A_phi_time_f(fit_sin_params[w][f][V][0], fit_sin_params[w][f][V][1], time, f) 
        ax2.plot(time, current*1e6, c='C'+str(i_V), linestyle='--', label='I' + str(V))
    
    plt.axvline(-fit_sin_params[w][f][V][1]/360/f+1/f,c='k')
    plt.axvline(-fit_sin_params[w][f][V][1]/360/f+2/f,c='k')
ax.legend()
ax2.legend()
    
for idx_f, f in enumerate(fs):
    ax = plt.subplot(2,2,idx_f+3)
    plt.title(str(f) + ' Hz')
    ax2 = ax.twinx()
    for i_V, V in enumerate(V_amps[f]):
        time = V_ames[f][V][:,0]
        ax.plot(time, V_ames[f][V][:,1], c='C'+str(i_V), alpha=0.5)
        current = sin_A_phi_time_f(fit_sin_params[w][f][V][0], fit_sin_params[w][f][V][1], time, f) 
        ax2.plot(time, current*1e6, c='C'+str(i_V), linestyle='--')
        
    ax.set_xlim((-fit_sin_params[w][f][V][1]/360/f+1/f+.9/f, -fit_sin_params[w][f][V][1]/360/f+1/f+1.1/f))
    ax.set_ylim((-0.1, 0.1))

plt.tight_layout()
plt.show()

### Fit sinusoids to V_ames

In [ ]:
def fit_sin2V_trace(x, time, f, V_trace):
    A   = x[0]
    phi = x[1]
    
    return np.sum((sin_A_phi_time_f(A, phi, time, f) - V_trace)**2)

np.random.seed(13454642)

# Fit.
fit_sin_params_V_ames = {f: {V: None for V in V_amps[f]} for f in fs}
for f  in fs:
    for V in V_amps[f]:
        time    = V_ames[f][V][:,0]
        V_trace = V_ames[f][V][:,1]
        
        A0 = np.max(np.abs((V_trace)))
        phi0 = (time[np.where(V_trace < 0)[0][0]] * f) * 180

        best_fit_loss = np.inf
        best_fit = None
        for _ in range(10):
            fit = minimize(
                fit_sin2V_trace, args=(time, f, V_trace),
                x0=[A0*np.random.uniform(0.5, 2), phi0*np.random.uniform(0.5, 2)],
                bounds=[(A0*0.5, A0*2), (-90, 90)]
            )

            if fit.fun < best_fit_loss:
                best_fit_loss = fit.fun
                best_fit = fit.x
                
        fit_sin_params_V_ames[f][V] = best_fit
        
        print("Phase difference = {:.2f} \t <= 0?".format(fit_sin_params_V_ames[f][V][1] - fit_sin_params['wo'][f][V][1]))

In [ ]:
data_utils.save_var(fit_sin_params_V_ames, 'data_preprocessed/V_ames_sinus_fits_params.pkl')

### Plot V_ames, raw and fits.

In [ ]:
for f in fs:
    plt.figure(figsize=(12,2))
    plt.title(str(w) + ' ' + str(f))
    for i_V, V in enumerate(V_amps[f]):
        time    = V_ames[f][V][:,0]
        V_trace = V_ames[f][V][:,1]

        plt.plot(time, V_trace, 'r', label='voltage' if V == 15 or V == 30 else '')
        plt.plot(
            time,
            sin_A_phi_time_f(fit_sin_params_V_ames[f][V][0], fit_sin_params_V_ames[f][V][1], time, f),
            'k', label='fit' if V == 15  or V == 30 else ''
        )
        plt.legend()

## Fit sinus to V_EDL

Compute V_EDL by subtracting V_ames from V_EDL

### Compute V_EDL

In [ ]:
V_EDL = {25: {}, 40: {}}

# Take only last interation and repeat
for f in fs:
    for V in V_amps[f]:
        time = np.arange(0,4000)*comsol_dt
        V_ames_temp = sin_A_phi_time_f(fit_sin_params_V_ames[f][V][0], fit_sin_params_V_ames[f][V][1], time, f)
        V_stim_temp = V*np.sin(2*np.pi*time*f)
        
        V_EDL[f][V] = np.concatenate([np.atleast_2d(time), np.atleast_2d(V_stim_temp - V_ames_temp)]).T

### Plot V_EDL

In [ ]:
plt.figure(figsize=(12,6))
for idx_f, f in enumerate(fs):
    plt.subplot(2,2,idx_f+1)
    plt.title("V_EDL @ " + str(f) + ' Hz')
    for V in V_amps[f]:
        plt.plot(V_EDL[f][V][:,0], V_EDL[f][V][:,1])

plt.tight_layout()
plt.show()

### Fit sinusoids to V_EDL data

In [ ]:
np.random.seed(123132)

# Fit.
fit_sin_params_V_EDL = {f: {V: None for V in V_amps[f]} for f in fs}
for f in fs:
    for V in V_amps[f]:
        time    = V_EDL[f][V][:,0]
        V_trace = V_EDL[f][V][:,1]
        
        A0 = np.max(np.abs((V_trace)))
        phi0 = (time[np.where(V_trace < 0)[0][0]] * f) * 180

        best_fit_loss = np.inf
        best_fit = None
        for _ in range(30):
            fit = minimize(
                fit_sin2V_trace, args=(time, f, V_trace),
                x0=[A0*np.random.uniform(0.5, 2), phi0*np.random.uniform(0.8, 1.3)],
                bounds=[(A0*0.5, A0*2), (-90, 90)],
            )

            if fit.fun < best_fit_loss:
                best_fit_loss = fit.fun
                best_fit = fit.x
                
        fit_sin_params_V_EDL[f][V] = best_fit

In [ ]:
data_utils.save_var(fit_sin_params_V_EDL, 'data_preprocessed/V_EDL_sinus_fits_params.pkl')

### Plot V_EDL, raw and fits.

In [ ]:
for f in fs:
    plt.figure(figsize=(12,2))
    plt.title(str(w) + ' ' + str(f))
    for i_V, V in enumerate(V_amps[f]):
        time    = V_EDL[f][V][:,0]
        V_trace = V_EDL[f][V][:,1]

        plt.plot(time, V_trace, 'r', label='voltage' if V == 15 or V == 30 else '')
        plt.plot(time, sin_A_phi_time_f(fit_sin_params_V_EDL[f][V][0], fit_sin_params_V_EDL[f][V][1], time, f),
                 'k', label='fit' if V == 15  or V == 30 else '')
        plt.legend()

# Compute phases of V_EDL

In [ ]:
EDL_phase_add = {} 
EDL_phase_total = {}

for f in fs:
    cur_phases = -np.array(list(fit_sin_params['wo'][f].values()))[:,1]
    V_phases   = -np.array(list(fit_sin_params_V_EDL[f].values()))[:,1]
 
    EDL_phase_add[f] = cur_phases
    EDL_phase_total[f] = -V_phases + cur_phases
    
data_utils.save_var(EDL_phase_total, 'data_preprocessed/EDL_phase_total.pkl')

In [ ]:
# Plot phases.
plt.figure(figsize=(12,3))
plt.title('Phis from sinus fits')
for f in fs:
    phis = EDL_phase_total[f]
    Vs = np.array(V_amps[f])/1000

    plt.plot(Vs, phis, '-*', label=str(w) + ' ' + str(f))
plt.legend()
plt.xlabel('V')
plt.ylabel('phase')
plt.show()

In [ ]:
# Plot currents vs. voltages.
plt.figure(figsize=(12,3))
plt.title('Amplitudes from sinus fits')
for f in fs:
    V_EDLs = np.array(list(fit_sin_params_V_EDL[f].values()))[:,0]
    Is     = np.array(list(fit_sin_params['wo'][f].values()))[:,0]

    plt.plot(V_EDLs, Is*1e6, '-*', label=str(f))
plt.legend()
plt.xlabel('V EDL [mV]')
plt.ylabel('current [uA]')
plt.show()

##### Estimate |Z| from sinus fits.

In [ ]:
absZ_est = {}
for f in fs:
    absZ_est[f] = {}
    for i_V, V in enumerate(V_amps[f]):
        V_EDL_i = fit_sin_params_V_EDL[f][V][0] /1000
        I_i     = fit_sin_params['wo'][f][V][0]
    
        absZ_est[f][V] = V_EDL_i/I_i
        
absZ_est

In [ ]:
data_utils.save_var(absZ_est, 'data_preprocessed/absZ_est.pkl')

# Use RC parallel circuit for electrode model

In [ ]:
def _s(n):
    return np.sin(-n*np.pi/2)

def _c(n):
    return np.cos(-n*np.pi/2)

def _t(phi):
    return 1/np.tan(phi/180*np.pi)

def _w(f):
    assert f in fs
    return (2*np.pi*f)

In [ ]:
def R_ges(R, C, f):
    return R / (1 + (_w(f)*C*R)**2)

In [ ]:
def X_ges(R, C, f):
    return (_w(f)*C*R**2) / (1 + (_w(f)*C*R)**2)

In [ ]:
def Z_ges(R, C, f):
    return np.sqrt(R_ges(R, C, f)**2 + X_ges(R, C, f)**2)

In [ ]:
def RC(phi, f):
    return -np.tan(phi/180*np.pi)/_w(f)

In [ ]:
def R_from_RC(absZ, RC, f):
    return absZ * np.sqrt(1 + _w(f)**2 * RC**2)

In [ ]:
RC_params = {}
for i_f, f in enumerate(fs):
    RC_params[f] = {}
    for i_V, V in enumerate(V_amps[f]):
        RC_params[f][V] = {}
        RCi = RC(EDL_phase_total[f][i_V], f)
        R = R_from_RC(absZ_est[f][V], RCi, f)
        C = RCi / R
        
        RC_params[f][V]['R'] = R
        RC_params[f][V]['C'] = C

data_utils.save_var(RC_params, 'data_preprocessed/RC_params.pkl')
RC_params